In [1]:
import multiprocessing
from multiprocessing import Pool
from functools import partial
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import matplotlib.pyplot as plt
from scipy import spatial
from scipy.stats import beta
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

In [4]:
a_alpha=1
b_alpha=400
a_beta=1
b_beta=800
a_betaf=3.5
b_betaf=800
a_gama=50
b_gama=200
a_teta0=10
b_teta0=1000
a_teta1=9000
b_teta1=300
P=1
U=2
K=2
J=1
n,T=100,360
epsln=0.001
hyper_params=np.array([a_alpha,b_alpha,a_beta,b_beta,a_betaf,b_betaf,a_gama,b_gama,a_teta0,b_teta0,a_teta1,b_teta1])

In [5]:
def Sample_alpha(a_alpha, b_alpha):
    for i in beta.rvs(a_alpha, b_alpha, size=10000):
        if (i>0.001)&(i<0.00501):
            alpha_=round(i,3)
            return alpha_ 
    if not(i>0.001):
        return -1.0
    else:
        return -2.0
    
def Sample_beta(a_beta, b_beta):
    for i in beta.rvs(a_beta, b_beta, size=10000):
        if (i>0.001)&(i<0.00501):
            beta_=round(i,4)
            return beta_ 
    if not(i>0.001):
        return -1.0
    else:
        return -2.0                

def Sample_betaf(a_betaf, b_betaf):
    for i in beta.rvs(a_betaf, b_betaf, size=1000):
        if (i>0.002)&(i<0.00501):
            betaf=round(i,4)
            return betaf
    if not(i>0.002):
        return -1.0
    else:
        return -2.0
def Sample_gama(a_gama,b_gama):
    for i in beta.rvs(a_gama, b_gama, size=10000):
        if (i>0.1)&(i<0.5):
            gama_=round(i,3)
            return gama_  
    if not(i>0.07):
        return -1.0
    else:
        return -2.0           

def Sample_theta0(a_teta0, b_teta0):
    for i in beta.rvs(a_teta0, b_teta0, size=10000):
        if (i>0.01)&(i<0.03):
            theta_0_=round(i,3)
            return theta_0_  
    if not(i>0.01):
        return -1.0
    else:
        return -2.0  

def Sample_theta1(a_teta1, b_teta1):
    for i in beta.rvs(a_teta1, b_teta1, size=10000):
        if i>0.799:
            theta_1_=round(i,3)
            return theta_1_ 
    if not(i>0.78):
        return -1

    
# function to sample infection and emission parameters(alpha,beta,betaf,gama,teta0,teta1)
def initialize_parameters(hyper_params):
    a_alpha=hyper_params[0]
    b_alpha=hyper_params[1]
    a_beta=hyper_params[2]
    b_beta=hyper_params[3]
    a_betaf=hyper_params[4]
    b_betaf=hyper_params[5]
    a_gama=hyper_params[6]
    b_gama=hyper_params[7]
    a_teta0=hyper_params[8]
    b_teta0=hyper_params[9]
    a_teta1=hyper_params[10]
    b_teta1=hyper_params[11]

    alpha_=Sample_alpha(a_alpha, b_alpha)
    while alpha_==-1:
        a_alpha=a_alpha+5
        hyper_params[0]=a_alpha
        alpha_=Sample_alpha(a_alpha, b_alpha)
    while alpha_==-2:
        b_alpha=b_alpha+500
        hyper_params[1]=b_alpha
        alpha_=Sample_alpha(a_alpha, b_alpha)
    beta_=Sample_beta(a_beta, b_beta)
    while beta_==-1:
        a_beta=a_beta+5
        hyper_params[2]=a_beta
        beta_=Sample_beta(a_beta, b_beta)
    while beta_==-2:
        b_beta=b_beta+500
        hyper_params[3]=b_beta
        beta_=Sample_beta(a_beta, b_beta)    
    betaf=Sample_betaf(a_betaf, b_betaf)
    while alpha_>beta_:
        alpha_=Sample_alpha(a_alpha, b_alpha)
        beta_=Sample_beta(a_beta, b_beta)
    while beta_>betaf:
        betaf=Sample_betaf(a_betaf, b_betaf)
    gama_=Sample_gama(a_gama,b_gama)
    theta_0_=Sample_theta0(a_teta0, b_teta0)
    while theta_0_==-1:
        a_teta0=a_teta0+5
        hyper_params[8]=a_teta0
        theta_0_=Sample_theta0(a_teta0, b_teta0)
    while theta_0_==-2:
        b_teta0=b_teta0+500
        hyper_params[9]=b_teta0
        theta_0_=Sample_theta0(a_teta0, b_teta0)       
    theta_1_=Sample_theta1(a_teta1, b_teta1)
    while theta_1_==-1:
        a_teta1=a_teta1+5
        hyper_params[8]=a_teta1
        theta_1_=Sample_theta1(a_teta0, b_teta0)
   
    params=np.array([alpha_,beta_,betaf,gama_,theta_0_,theta_1_])
    return params


In [6]:
def CNbr(G,X):
    n,T=X.shape[0],X.shape[1]
    C=np.zeros((T,n))
    for t in range(T):
        C[t]=G[t].dot(X.T[t])
    return C.T

In [7]:
params=initialize_parameters(hyper_params)
params

array([0.001 , 0.0014, 0.0038, 0.178 , 0.016 , 0.968 ])

In [8]:
# Generate synthetic data,G ,Y:
n,T,y=100,360,5
U,K=2,2
P=1
number_families=15
#synthetic_data0=Synthetic_Data(n,T,y,params,number_families)
#%store synthetic_data0

In [9]:
m=[1,2,3,4,5,6,12,52,360]
for i in np.random.poisson(m[0], 10000):
    if i>0:
        mu=i
        break
mu        

2

In [10]:
# function to generate missing data:
# mu is the average number of tests for a typical family during a year:
m=[1,2,3,4,5,6,12,52]
def missing_data(m,YF):
    number_of_families,T=YF.shape[0],YF.shape[1]
    YF_missing=np.zeros((number_of_families,T))-1
    time=list(range(T))
    random.shuffle(time)
    for f in range(number_of_families):
        for i in np.random.poisson(m, 10000):
            if i>0:
                mu=i
                break
        idx=np.sort(np.random.choice(time, mu,replace=False))
        for t in idx:
            YF_missing[f,t]=YF[f,t]
        
    return YF_missing 

In [11]:
import os
G=np.load('G.npy')
YF=np.load('YF.npy')
X=np.load("X.npy")
F=np.load('F.npy')

In [13]:
import Code_mu
data_inputs=[hyper_params,hyper_params,hyper_params,hyper_params,hyper_params,hyper_params,hyper_params,hyper_params]

if __name__ == '__main__':
    pool = Pool(8)                         # Create a multiprocessing Pool
    pool_list=pool.map(Code_mu.process, data_inputs)

In [14]:
l=[]
for i in range(8):
    l.append([pool_list[i][0],pool_list[i][1],missing_data(m[i],YF)])

In [ ]:
import workers_mu
if __name__ ==  '__main__': 
    with Pool(processes =8) as pool:

        parallel_output = pool.starmap(workers_mu.algrthm,l )# use tqdm to show the progress
        pool.close()
        pool.join()


In [ ]:
def plot_ROC(y_score,y_test,i):
    plt.figure()
    lw = 2
    y_score=np.hstack(y_score)
    y_test=np.hstack(y_test)
    fpr, tpr, thresholds = roc_curve(y_test, y_score,pos_label=1)
    roc_auc = auc(fpr, tpr)
    plt.plot(
        fpr,
        tpr,
        color="darkorange",
        lw=lw,
        label="ROC curve (area = %0.2f)" % roc_auc,
    )
    plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC for family test result problem")
    plt.legend(loc="lower right")
    plt.savefig(str(i)+".pdf")
    plt.show()
    return roc_auc

unique_rows = np.unique(F, axis=0)

In [ ]:
# plot figure 1 for different values of missing data:
cnt=0
for i in parallel_output:
    plot_ROC(X,i,cnt)
    cnt=cnt+1